In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic' # Windows
import seaborn as sns

from sklearn.model_selection import train_test_split

### Data : 캘리포니아 주택 가격
- longitude : 경도
- latitude : 위도
- housing_median_age : 주택 연수(중앙값)
- total_rooms : 전체 방 수
- total_bedrooms : 전체 침실 수
- population : 인구 수
- households : 세대 수
- median_income : 소득(중앙값)
- median_house_value : 주택 가치(중앙값)
- ocean_proximity : 바다 접근도

In [ ]:
housing = pd.read_csv('./data/housing.csv')
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.isnull().sum()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

In [ ]:
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
test_set.head()

In [ ]:
housing['median_income'].hist()

In [ ]:
import numpy as np

housing['income_cat'] = pd.cut(housing['median_income'], bins=[0., 1.5, 3., 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])

In [ ]:
housing['income_cat'].value_counts()

In [ ]:
housing.plot(kind='scatter', 
             x='longitude',
             y='latitude', 
             alpha=0.4,     # 투명도 
             s=housing['population']/100,   # 점 크기 
             label='population',    # 범례 라벨 : 인구
             figsize=(10, 7), 
             c='median_house_value',    # 색상 : 중간 주택 가격
             cmap=plt.get_cmap('jet'),  # 색상 맵 : jet 
             colorbar=True, # 색상 바 표시
             sharex=False   # x축 공유 안함
             )

In [ ]:
# 2. 결측치 제거
housing = housing.dropna()

In [ ]:
# 3. 범주형 변수 One-hot encoding
housing = pd.get_dummies(housing, columns=["ocean_proximity"], dtype="int")

In [ ]:
housing

In [ ]:
# 4. 타겟 컬럼 제외 (우리는 clustering이니까 정답은 없음)
X = housing.drop("median_house_value", axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
# 5. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.cluster import KMeans

# KMeans 클러스터링
k = 5  # 군집 개수 (조정 가능)
kmeans = KMeans(n_clusters=k, random_state=42, max_iter=500)
housing['cluster'] = kmeans.fit_predict(X_scaled)

In [ ]:
# 7. 결과 확인
print(housing['cluster'].value_counts())

In [ ]:
from sklearn.decomposition import PCA

# 8. 차원 축소(PCA)로 시각화 (2D)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)



In [ ]:
housing['pca1'] = X_pca[:, 0]
housing['pca2'] = X_pca[:, 1]

In [ ]:
# 9. 시각화
plt.figure(figsize=(10, 6))
sns.scatterplot(data=housing, x='pca1', y='pca2', hue='cluster', palette='tab10')
plt.title("KMeans Clustering of Housing Data (PCA reduced)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.grid(True)
plt.show()

#### (실습) K 값 산정 

In [ ]:
# 시각화


#### (실습) K 값 적용 신규 적용

In [ ]:
kmeans = KMeans(n_clusters = 5)
kmeans.fit(X_scaled)

#### (실습) 실루엣 계수 시각화 적용 

### 분석 및 해석 방법

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('./data/boston.csv')
data.head()

- crim : 주변 범죄율
- zn : 25,000 평방피트를 초과 거주지역 비율
- indus : 비소매 상업지역 면적비율
- chas : 경계선 강 유무(강의 경계 1, 아니면 0)
- nox : 일산화질소 농도
- rm : 평균 방 개수
- age : 1940년 이전에 건설된 주택의 비율
- dis : 고용 센터와의 거리에 따른 가중치
- rad : 고속도로와의 접근성(방사형) 지수
- tax : 재산세율
- ptratio : 학생/교사 비율
- black : 흑인의 비율
- lstat : 빈곤층 비율
- medv : 보스턴 506개 타운의 주택 가격 중앙값(1978년 기준)

In [ ]:
del data['Unnamed: 0']
data.head()

In [ ]:
data['chas'].value_counts()  # 범주형 데이터로 판단됨

In [ ]:
del data['chas']

In [ ]:
# 타겟변수를 복사해 두고, 타겟변수 컬럼 삭제
medv = data['medv']
del data['medv']

- 차원축소(PCA) 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# 정규화
scaler = StandardScaler()
scaler.fit(data)
scaler_data = scaler.transform(data)

In [ ]:
# 주성분 분석 
pca = PCA(n_components = 2)
pca.fit(scaler_data)

In [ ]:
data2 = pd.DataFrame(data = pca.transform(scaler_data), columns=['pc1', 'pc2'])

In [ ]:
data2.head()

### 군집의 개수 정하기

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
x = []   # k 가 몇개인지 
y = []   # 응집도가 몇인지 

for k in range(1, 30):
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(data2)
    
    x.append(k)
    y.append(kmeans.inertia_)

In [ ]:
plt.plot(x, y)

In [ ]:
kmeans_model = KMeans(n_clusters=4)
kmeans_model.fit(data2)
data2['labels'] = kmeans_model.labels_
#data2['labels'] = kmeans_model.transform(data2)

In [ ]:
data2.head()

In [ ]:
sns.scatterplot(x='pc1', y='pc2', hue='labels', data=data2)

### 결과 해석

- 어떤 그룹의 집 값이 가장 높을까? 평균으로 비교

In [ ]:
data2['medv'] = medv

In [ ]:
data2.head()

In [ ]:
data2[data2['labels']==0]['medv'].mean()

In [ ]:
medv_list = []

for i in range(4):
    medv_avg = data2[data2['labels']==i]['medv'].mean()
    medv_list.append(medv_avg)

In [ ]:
print(medv_list)

In [ ]:
sns.barplot(x=['group_0', 'group_1', 'group_2', 'group_3'], y=medv_list)

- 최상위 그룹과 최하위 그룹을 비교하여 집값의 평균이 높거나 낮은 이유에 대하여 확인

In [ ]:
data['labels'] = data2['labels']

In [ ]:
group = data[(data['labels']==1) | (data['labels']==3)]

In [ ]:
group = group.groupby('labels').mean().reset_index()

In [ ]:
group

In [ ]:

del group['labels']

In [ ]:
group.columns = ['범죄율', '주택지', '농지면적', '산화질소농도', '방개수', '1940년 이전에 건설', '센터와의 거리', '접근성', '재산세', '학생 교사 비율', '흑인비율', '빈곤층비율']


In [ ]:
group

In [ ]:
columns = list(group.columns)

In [ ]:
del columns[0]


In [ ]:
column = group.columns
fig, ax = plt.subplots(2, 6, figsize=(30, 13))

for i in range(11):
    sns.barplot(x = columns[i], y= column[i], data=group, ax=ax[i//6, i%6])

#### 기본적인 분석 예시

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 예제 데이터
#X = df[['var1', 'var2', 'var3', 'var4']]
#X = home_data.loc[:, ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'population', 'households']]
X = data

# 스케일링 필수 (PCA는 분산 기반이므로)
X_scaled = StandardScaler().fit_transform(X)

# PCA 2개 컴포넌트
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 주성분 로딩 (변수 기여도)
loadings = pd.DataFrame(pca.components_.T,  # 행: 변수 / 열: PC1, PC2
                        columns=['PC1', 'PC2'],
                        index=X.columns)

# ▶️ 로딩 플롯 그리기
plt.figure(figsize=(6,6))
plt.axhline(0, color='grey', lw=1)
plt.axvline(0, color='grey', lw=1)
for var in loadings.index:
    plt.arrow(0, 0, loadings.loc[var, 'PC1'], loadings.loc[var, 'PC2'],
              head_width=0.03, color='blue')
    plt.text(loadings.loc[var, 'PC1']*1.1, loadings.loc[var, 'PC2']*1.1,
             var, color='red')

plt.title("Loading Plot (변수 기여도)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid()
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# PCA 결과를 기반으로 KMeans 클러스터링
kmeans = KMeans(n_clusters=4, random_state=42)
cluster_labels = kmeans.fit_predict(X_pca)

# 시각화
pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
pca_df['Cluster'] = cluster_labels

sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="Cluster", palette='Set2')
plt.title("K-Means Clustering on PCA Components")
plt.show()